In [1]:
################################################
###
### Here we mainly collect features from the cropped and segemented spine cord in mouse embryo 
### 
################################################

In [2]:
import pandas as pd
import numpy as np

from skimage import measure
from skimage.filters import threshold_otsu, rank
from skimage.io import imread
from skimage.io import imsave
from skimage import filters
from skimage import morphology
from skimage.data import cells3d
from skimage.measure import label


from pyclesperanto_prototype import imshow
import pyclesperanto_prototype as cle

import gc

from itertools import chain
# define nones functions
nones = lambda n: [None for _ in range(n)]

from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt
import stackview


In [3]:
## in case to run script 
from sys import argv, exit
import os

In [4]:
## test https://github.com/haesleinhuepf/spheri/blob/2427867b86b12e814c49ee80a0dc68ac3371b99f/demo/demo.ipynb
## original code doesn't work for some reasons, dissect the functions
DEFAULT_SMOOTH_ITERATIONS = 15

def sphericity_wadell(volume, surface_area):
    # "Hakon Wadell defined sphericity as the surface area of a sphere of the same volume as the particle divided by the actual surface area of the particle."
    # https://en.wikipedia.org/wiki/Sphericity
    # https://www.journals.uchicago.edu/doi/10.1086/624298
    import math
    return pow(math.pi, 1/3) * pow(6 * volume, 2/3) / surface_area

def sphericity_legland(volume, surface_area):
    # https://imagej.net/plugins/morpholibj#shape-factors
    import math
    return 36 * math.pi * pow(volume, 2) / pow(surface_area, 3)

def solidity(surface_volume, convex_hull_volume):
    return surface_volume / convex_hull_volume

def surface_meshes(label_image, smooth_iterations:int=DEFAULT_SMOOTH_ITERATIONS):
    import numpy as np
    import vedo

    result = {}
    for label in np.unique(label_image):
        if label == 0: # skip background
            continue
        
        binary_image = np.asarray((label_image == label) * 1)
        
        extended_binary_image = np.zeros([s + 2 for s in binary_image.shape])
        extended_binary_image[1:-1,1:-1,1:-1] = binary_image
        
        volume = vedo.Volume(extended_binary_image)
        surface = volume.isosurface(value=0.5)

        surface = surface.clean()

        # might work but takes very long time:
        # surface = surface.smooth_mls_2d()
        # might work but takes long time:
        if smooth_iterations > 0:
            surface = surface.smooth(niter=smooth_iterations)

        result[label] = surface
    
    return result

def soli_measure(label_image, smooth_iterations:int=DEFAULT_SMOOTH_ITERATIONS):
    import numpy as np
    import pandas as pd
    import math
    import vedo
    
    result = {
        "label":[],
        "surface_area":[],
        "volume":[],
        "convex_hull_area":[],
        "convex_hull_volume":[],
        "solidity":[],
        "sphericity_wadell":[],
        "sphericity_legland":[],
    }
    
    for label, surface in surface_meshes(label_image, smooth_iterations=smooth_iterations).items():
        
        convex_hull = vedo.shapes.ConvexHull(surface)

        surface_area = surface.area()
        surface_volume = surface.volume()
        
        convex_hull_area = convex_hull.area()
        convex_hull_volume = convex_hull.volume()
        
        result["label"].append(label)
        result["surface_area"].append(surface_area)
        result["volume"].append(surface_volume)
        result["convex_hull_area"].append(convex_hull_area)
        result["convex_hull_volume"].append(convex_hull_volume)
        result["solidity"].append(solidity(surface_volume, convex_hull_volume))
        result["sphericity_wadell"].append(sphericity_wadell(surface_volume, surface_area))
        result["sphericity_legland"].append(sphericity_legland(surface_volume, surface_area))

    return pd.DataFrame(result)


In [4]:
#CPoutDir = "/Volumes/groups/tanaka/People/current/jiwang/projects/RA_competence/images_data/d4_WT"
#WTDir = "/Volumes/groups/tanaka/People/current/jiwang/projects/RA_competence/images_data/d4_WT" 
inputDir = "/groups/tanaka/biooptics/Hannah/CellPose_Embryos/CellPose_Jingkui/embH-for-Jingkui"
fileName = '250713_30xsil-041umZ_E85-embH_crop-somite4'
#fileName = '211210_d6_RAd2_D2_49_01_isotropic'

#inputDir = "/groups/tanaka/People/current/jiwang/projects/RA_competence/images_data/test_WTd6"
#fileName = '20220727_d6_10x_RA-18h_E2_121_1981_isotropic'
#fileName = "20220726_d4_10x_RA-18h_E2_121_1745_isotropic"
#fileName = '241023_10x_d4-2umZ_4-PS-WG_RA_FA2-Px6_F9_95_01_isotropic'
#fileName = '211209_d3_RAd2_D2_51_01_isotropic'
#fileName = '211210_d6_RAd2_D2_49_01_isotropic'

path_results = '/groups/tanaka/People/current/jiwang/projects/RA_competence/images_data/results/' \
        + 'embryo_features' + '/'
outDir = path_results

if not os.path.exists(outDir):
    os.mkdir(outDir)
    
print(outDir)


/groups/tanaka/People/current/jiwang/projects/RA_competence/images_data/results/embryo_features/


In [5]:
mask = imread(os.path.join(inputDir, str(fileName + "_blur_cp_masks_cleaned.tif"))) # segmented masks by cellpose and manually cropped 

In [6]:
mask.shape

(282, 577, 1203)

In [7]:
labels_mask, nb_mask = measure.label(mask, return_num = True)
nb_mask

1200

In [8]:
crops = imread(os.path.join(inputDir, str(fileName + ".tif")))

In [9]:
crops.shape

(282, 577, 1203, 4)

In [13]:
#xx =  crops[:, 0:576, :, :]

In [14]:
#xx.shape 

(282, 576, 1203, 4)

In [15]:
foxa2 = xx[:, :, :, 0] 
pax6 = xx[:, :, :, 1]
sox2 = xx[:, :, :, 2]
dapi = xx[:, :, :, 3]

In [40]:
imsave(os.path.join("/groups/tanaka/People/current/jiwang/projects/RA_competence/images_data/CellPose_Embryos", str(fileName + ".tif")), xx)

/tmp/ipykernel_46191/2306378546.py:1: UserWarning: /groups/tanaka/People/current/jiwang/projects/RA_competence/images_data/CellPose_Embryos/250713_30xsil-041umZ_E85-embH_crop-somite4.tif is a low contrast image
  imsave(os.path.join("/groups/tanaka/People/current/jiwang/projects/RA_competence/images_data/CellPose_Embryos", str(fileName + ".tif")), xx)


In [16]:
foxa2.shape, pax6.shape, sox2.shape, dapi.shape

((282, 576, 1203), (282, 576, 1203), (282, 576, 1203), (282, 576, 1203))

In [23]:
labels_mask, nb_mask = measure.label(mask, return_num = True)
nb_mask

880

In [16]:
sphere_stats = soli_measure(labels_mask)

KeyboardInterrupt: 

In [12]:
sphere_stats

,label,surface_area,volume,convex_hull_area,convex_hull_volume,solidity,sphericity_wadell,sphericity_legland
0,1,13544.056027,100812.412015,12346.936064,117088.157321,0.860996,0.773413,0.462630
1,2,7209.055321,40920.241630,6631.511135,46770.142215,0.874922,0.796583,0.505468
2,3,8764.981666,54123.336913,7973.752629,61960.160939,0.873518,0.789447,0.492004
3,4,7774.917034,49102.203707,7092.193546,53407.888232,0.919381,0.834044,0.580186
4,5,9086.373952,63318.882704,8285.943791,68304.650009,0.927007,0.845504,0.604431
5,6,8844.748510,59814.370151,8189.584035,65843.547899,0.908432,0.836249,0.584799
6,7,10656.102786,73474.770165,9772.003613,83479.801976,0.880150,0.796119,0.504584
7,8,7727.444511,49492.468261,7136.790221,54483.847341,0.908388,0.843609,0.600375
8,9,3498.423320,14581.595253,3266.378652,16420.502556,0.888012,0.825053,0.561624
9,10,12885.920174,95531.584402,11742.351466,111907.184179,0.853668,0.784272,0.482392


In [ ]:
#print(sphere_stats[["label", "surface_area",  "volume", "convex_hull_area", "convex_hull_volume", "solidity", "sphericity_wadell", "sphericity_legland"]])

In [18]:
## mask image 
props_mask = measure.regionprops_table(
    labels_mask, 
    #intensity_image = C3,
    properties=('label', 'area', 'equivalent_diameter_area', 'centroid', 'axis_major_length', 'axis_minor_length', 'euler_number'),
)
props_mask = pd.DataFrame(props_mask)
props_mask = props_mask.add_suffix('_mask')

## C1
props_C1 = measure.regionprops_table(
    labels_mask, 
    intensity_image = C1,
    properties=('intensity_max', 'intensity_mean', 'intensity_min', 'intensity_std'),
)
props_C1 = pd.DataFrame(props_C1)
props_C1 = props_C1.add_suffix('_C1')

## C2
props_C2 = measure.regionprops_table(
    labels_mask, 
    intensity_image = C2,
    properties=('intensity_max', 'intensity_mean', 'intensity_min', 'intensity_std'),
)
props_C2 = pd.DataFrame(props_C2)
props_C2 = props_C2.add_suffix('_C2')

props_C3 = measure.regionprops_table(
    labels_mask, 
    intensity_image = C3,
    properties=('intensity_max', 'intensity_mean', 'intensity_min', 'intensity_std'),
)
props_C3 = pd.DataFrame(props_C3)
props_C3 = props_C3.add_suffix('_C3')

props_C4 = measure.regionprops_table(
    labels_mask, 
    intensity_image = C4,
    properties=('intensity_max', 'intensity_mean', 'intensity_min', 'intensity_std'),
)
props_C4 = pd.DataFrame(props_C4)
props_C4 = props_C4.add_suffix('_C4')

,image,cyst_index,cyst_size,pct_c1,pct_c2,pct_c3,pct_c4,cutoff_c1,cutoff_c2,cutoff_c3,cutoff_c4
0,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,1,100898,0.272919,0.275060,0.131479,0.439404,2540,4251,928,5116
1,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,2,40980,0.205905,0.298121,0.083260,0.477672,2270,4000,712,4248
2,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,3,54192,0.254890,0.262696,0.115035,0.496826,2007,3980,1045,6074
3,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,4,49162,0.070237,0.360502,0.022619,0.504699,1968,6639,1362,5480
4,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,5,63382,0.303951,0.660803,0.400934,0.527326,4098,1343,1844,5950
5,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,6,59876,0.341021,0.138002,0.149509,0.437204,2337,3239,935,3173
6,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,7,73552,0.281678,0.172667,0.095565,0.439227,2759,4695,1156,5535
7,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,8,49552,0.404383,0.386180,0.172808,0.459537,3558,1543,1080,4238
8,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,9,14624,0.292738,0.201723,0.331783,0.500068,3472,2312,1387,4897
9,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,10,95612,0.066174,0.381563,0.049669,0.447789,1450,4575,822,5041


In [25]:
cyst_index = np.where(labels_mask == 0)
cyst_index[0].size
cyst_index

(array([ 0,  0,  0, ..., 90, 90, 90]),
 array([  0,   0,   0, ..., 498, 498, 498]),
 array([  0,   1,   2, ..., 496, 497, 498]))

In [26]:
np.unique(labels_mask)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

In [31]:
data = {"image": [], "cyst_index": [], "cyst_size": [],  "pct_c1": [], "pct_c2": [], "pct_c3": [], "pct_c4": [],  
        "cutoff_c1": [],  "cutoff_c2": [],  "cutoff_c3": [],  "cutoff_c4": []}
df = pd.DataFrame(data)

for i in np.unique(labels_mask):
    if i == 0: # skip background
        continue
    
    cyst_index = np.where(labels_mask == i)
    print(str(i) + '--' + str(cyst_index[0].size))
                
    crop_c1 = C1[cyst_index]
    crop_c2 = C2[cyst_index]
    crop_c3 = C3[cyst_index]
    crop_c4 = C4[cyst_index]
                    
    cutoff_c1 = threshold_otsu(crop_c1)
    cutoff_c2 = threshold_otsu(crop_c2)
    cutoff_c3 = threshold_otsu(crop_c3)
    cutoff_c4 = threshold_otsu(crop_c4)
    
    index_c1 = np.where(crop_c1 > cutoff_c1)
    index_c2 = np.where(crop_c2 > cutoff_c2)
    index_c3 = np.where(crop_c3 > cutoff_c3)
    index_c4 = np.where(crop_c4 > cutoff_c4)
                
    df.loc[len(df)] = [fileName, i,  cyst_index[0].size, 
                       index_c1[0].size/cyst_index[0].size, 
                       index_c2[0].size/cyst_index[0].size,
                       index_c3[0].size/cyst_index[0].size,
                       index_c4[0].size/cyst_index[0].size,
                       cutoff_c1, cutoff_c2, cutoff_c3, cutoff_c4
                      ]

1--100898
2--40980
3--54192
4--49162
5--63382
6--59876
7--73552
8--49552
9--14624
10--95612
11--49820
12--656
13--17436
14--72308
15--155180
16--13152
17--1624
18--5932
19--16
20--404
21--28
22--77304
23--32152
24--1472
25--9168
26--4424
27--264
28--25096
29--840


In [33]:
df = pd.concat([df, sphere_stats, props_mask, props_C1, props_C2, props_C3, props_C4 ], axis=1)

In [34]:
df

,image,cyst_index,cyst_size,pct_c1,pct_c2,pct_c3,pct_c4,cutoff_c1,cutoff_c2,cutoff_c3,...,intensity_min_C2,intensity_std_C2,intensity_max_C3,intensity_mean_C3,intensity_min_C3,intensity_std_C3,intensity_max_C4,intensity_mean_C4,intensity_min_C4,intensity_std_C4
0,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,1,100898,0.272919,0.275060,0.131479,0.439404,2540,4251,928,...,244.0,2474.914649,3617.0,510.150142,136.0,446.968312,13953.0,4871.923646,349.0,2386.967747
1,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,2,40980,0.205905,0.298121,0.083260,0.477672,2270,4000,712,...,233.0,2432.756541,2769.0,386.441484,157.0,257.884074,10497.0,4177.835993,309.0,1891.627846
2,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,3,54192,0.254890,0.262696,0.115035,0.496826,2007,3980,1045,...,246.0,2516.811095,3843.0,511.632473,150.0,504.040937,15409.0,6059.521055,340.0,2834.690856
3,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,4,49162,0.070237,0.360502,0.022619,0.504699,1968,6639,1362,...,298.0,3908.333593,4617.0,398.542289,157.0,348.352901,14366.0,5501.674301,411.0,2628.938949
4,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,5,63382,0.303951,0.660803,0.400934,0.527326,4098,1343,1844,...,334.0,467.238979,4755.0,1672.855006,210.0,986.055890,15044.0,6083.195197,542.0,2827.865248
5,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,6,59876,0.341021,0.138002,0.149509,0.437204,2337,3239,935,...,189.0,1626.494800,3829.0,555.088099,141.0,457.178277,7038.0,3044.188439,240.0,1218.934757
6,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,7,73552,0.281678,0.172667,0.095565,0.439227,2759,4695,1156,...,209.0,2567.735001,4498.0,559.860752,138.0,504.910897,16621.0,5257.254772,244.0,2723.336593
7,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,8,49552,0.404383,0.386180,0.172808,0.459537,3558,1543,1080,...,229.0,674.588492,3772.0,677.806042,156.0,545.737409,10678.0,4113.750565,334.0,1827.208130
8,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,9,14624,0.292738,0.201723,0.331783,0.500068,3472,2312,1387,...,234.0,1078.742579,4239.0,1152.895377,175.0,765.750619,11486.0,4897.475451,412.0,2256.506213
9,20220727_d6_10x_RA-18h_E2_121_1981_isotropic,10,95612,0.066174,0.381563,0.049669,0.447789,1450,4575,822,...,200.0,2683.378277,4049.0,362.918954,135.0,265.881142,14142.0,4836.499990,304.0,2332.196886


In [37]:
df.to_csv(os.path.join(outDir, "parameterCollection_cyst_size_sphericity_intensity_voxels.csv"), index=True, header=True)

In [4]:
########################################
###
### here batch process files in a specific folder
###
########################################

In [12]:
imageDir ="/groups/tanaka/People/current/jiwang/projects/RA_competence/images_data/CellPose_Embryos/test"

outDir = '/groups/tanaka/People/current/jiwang/projects/RA_competence/images_data/results/' \
        + 'embryo_features' + '/'

if not os.path.exists(outDir):
    os.mkdir(outDir)
    
print(outDir)

/groups/tanaka/People/current/jiwang/projects/RA_competence/images_data/results/embryo_features/


In [22]:
image = "/groups/tanaka/People/current/jiwang/projects/RA_competence/images_data/CellPose_Embryos/test/250713_30xsil-041umZ_E85-embH_crop-somite4.tif"
print(os.path.basename(image))
os.path.dirname(image)

250713_30xsil-041umZ_E85-embH_crop-somite4.tif


'/groups/tanaka/People/current/jiwang/projects/RA_competence/images_data/CellPose_Embryos/test'

In [13]:
import pandas as pd
from skimage import measure
from skimage.morphology import disk
from skimage.filters import threshold_otsu, rank
from skimage import measure
from skimage.morphology import disk
from skimage.filters import threshold_otsu, rank
import pyclesperanto_prototype as cle
from skimage import filters
from skimage.filters import try_all_threshold
import napari_simpleitk_image_processing as nsitk

In [14]:
for nm in os.listdir(imageDir):
    #if "C4_CystMask.tiff" in nm:
    if "_blur_cp_masks_cleaned.tif" in nm:
        #print(nm)
        #fileName = nm.replace('_C4_CystMask.tiff','')
        fileName = nm.replace('_blur_cp_masks_cleaned.tif','')
        
        mask = imread(os.path.join(imageDir, str(fileName + "_blur_cp_masks_cleaned.tif"))) # mask segmeted cysts
        crops = imread(os.path.join(imageDir, str(fileName + ".tif"))) 
        print(fileName, mask.shape, crops.shape)
        
        foxa2 = crops[:, :, :, 0] 
        pax6 = crops[:, :, :, 1]
        sox2 = crops[:, :, :, 2]
        dapi = crops[:, :, :, 3]
        
        labels_mask, nb_mask = measure.label(mask, return_num = True)
        
        ## sphericity parameters
        #sphere_stats = soli_measure(labels_mask)
        
        
        ## mask and foxa2 channel features
        props_C1 = measure.regionprops_table(
            labels_mask, 
            intensity_image = foxa2,
            properties=('label', 'area', 'equivalent_diameter_area', 'centroid', 'intensity_max', 'intensity_mean', 'intensity_min', 'intensity_std'))
        props_C1 = pd.DataFrame(props_C1)
        props_C1 = props_C1.add_suffix('_foxa2')
        
        ## C2
        props_C2 = measure.regionprops_table(labels_mask, 
                                             intensity_image = pax6, 
                                             properties=('intensity_max', 'intensity_mean', 'intensity_min', 'intensity_std'))
        props_C2 = pd.DataFrame(props_C2)
        props_C2 = props_C2.add_suffix('_pax6')
        
        # channel C3
        props_C3 = measure.regionprops_table(
            labels_mask, 
            intensity_image = sox2,
            properties=('intensity_max', 'intensity_mean', 'intensity_min', 'intensity_std'),
            )
        props_C3 = pd.DataFrame(props_C3)
        props_C3 = props_C3.add_suffix('_sox2')
        
        # chanel 4
        props_C4 = measure.regionprops_table(
            labels_mask, 
            intensity_image = dapi,
            properties=('intensity_max', 'intensity_mean', 'intensity_min', 'intensity_std'),
            )
        props_C4 = pd.DataFrame(props_C4)
        props_C4 = props_C4.add_suffix('_dapi')
        
        
        df = pd.concat([props_C1, props_C2, props_C3, props_C4], axis=1)
        #df = pd.concat([sphere_stats, props_C1, props_C2, props_C3, props_C4], axis=1)
        
        df.to_csv(os.path.join(outDir, "featuresCollection_" + fileName + ".csv"), index=True, header=True)
        
        #break            

250713_30xsil-041umZ_E85-embH_crop-somite4 (282, 577, 1203) (282, 577, 1203, 4)


In [15]:
imageDir, outDir

('/groups/tanaka/People/current/jiwang/projects/RA_competence/images_data/CellPose_Embryos/test',
 '/groups/tanaka/People/current/jiwang/projects/RA_competence/images_data/results/embryo_features/')

In [16]:
print(fileName)

250713_30xsil-041umZ_E85-embH_crop-somite4


In [ ]:
########################################
## counting the FoxA2+ and Pax6
########################################

In [7]:
imageDir ="/groups/tanaka/People/current/jiwang/projects/RA_competence/images_data/CellPose_Embryos/"

outDir = '/groups/tanaka/People/current/jiwang/projects/RA_competence/images_data/results/' \
        + 'embryo_features' + '/'

if not os.path.exists(outDir):
    os.mkdir(outDir)
    
print(outDir)

/groups/tanaka/People/current/jiwang/projects/RA_competence/images_data/results/embryo_features/


In [8]:
import pandas as pd
from skimage import measure
from skimage.filters import threshold_otsu, rank

data = {"image": [], "cyst_index": [], "cyst_size": [],  
        "cutoff_otsu_global_foxa2": [],  "cutoff_otsu_global_pax6": [], "nb_foxa2_otsu_global": [], "nb_pax6_otsu_global": [],
        "cutoff_li_global_foxa2": [],  "cutoff_li_global_pax6": [], "nb_foxa2_li_global": [], "nb_pax6_li_global": [],
        "cutoff_mean_global_foxa2": [],  "cutoff_mean_global_pax6": [], "nb_foxa2_mean_global": [], "nb_pax6_mean_global": [],
       }
df = pd.DataFrame(data)
df

,image,cyst_index,cyst_size,cutoff_otsu_global_foxa2,cutoff_otsu_global_pax6,nb_foxa2_otsu_global,nb_pax6_otsu_global,cutoff_li_global_foxa2,cutoff_li_global_pax6,nb_foxa2_li_global,nb_pax6_li_global,cutoff_mean_global_foxa2,cutoff_mean_global_pax6,nb_foxa2_mean_global,nb_pax6_mean_global


In [9]:
for nm in os.listdir(imageDir):
    #if "C4_CystMask.tiff" in nm:
    if "_blur_cp_masks_cleaned.tif" in nm:
        #print(nm)
        #fileName = nm.replace('_C4_CystMask.tiff','')
        fileName = nm.replace('_blur_cp_masks_cleaned.tif','')
        
        data = {"image": [], "cyst_index": [], "cyst_size": [],  
        "cutoff_otsu_global_foxa2": [],  "cutoff_otsu_global_pax6": [], "nb_foxa2_otsu_global": [], "nb_pax6_otsu_global": [],
        "cutoff_li_global_foxa2": [],  "cutoff_li_global_pax6": [], "nb_foxa2_li_global": [], "nb_pax6_li_global": [],
        "cutoff_mean_global_foxa2": [],  "cutoff_mean_global_pax6": [], "nb_foxa2_mean_global": [], "nb_pax6_mean_global": []}
        
        df = pd.DataFrame(data)
        
        mask = imread(os.path.join(imageDir, str(fileName + "_blur_cp_masks_cleaned.tif"))) # mask segmeted cysts
        crops = imread(os.path.join(imageDir, str(fileName + ".tif"))) 
        print(fileName, mask.shape, crops.shape)
        
        foxa2 = crops[:, :, :, 0] 
        pax6 = crops[:, :, :, 1]
        sox2 = crops[:, :, :, 2]
        dapi = crops[:, :, :, 3]
        
        labels_mask, nb_cyst = measure.label(mask, return_num = True)
        
        cutoff_global_otsu_foxa2 = threshold_otsu(foxa2)
        cutoff_global_otsu_pax6 = threshold_otsu(pax6)
        cutoff_global_li_foxa2 = filters.threshold_li(foxa2)
        cutoff_global_li_pax6 = filters.threshold_li(pax6)
        cutoff_global_mean_foxa2 = filters.threshold_mean(foxa2)
        cutoff_global_mean_pax6 = filters.threshold_mean(pax6)
        
        for i in range(nb_cyst+1):
            if i > 0:
                #print(i) # prints: -1, 0, 1, 2, 3, 4,
                cyst_index = np.where(labels_mask == i)
                
                keep = [fileName, i, cyst_index[0].size]
                
                foxa2_cyst = foxa2[cyst_index]
                pax6_cyst = pax6[cyst_index]
                                
                ## define the FoxA2+ only, Pax6+ only, and double positive with cyst otsu
                xx = np.where(foxa2_cyst > cutoff_global_otsu_foxa2) #  
                yy = np.where(pax6_cyst > cutoff_global_otsu_pax6)
                
                keep = keep + [cutoff_global_otsu_foxa2, cutoff_global_otsu_pax6, xx[0].size, yy[0].size]
                
                ## cutoff of FoxA2 and Pax6 in each cyst with li thresholding
                xx = np.where(foxa2_cyst > cutoff_global_li_foxa2) #  
                yy = np.where(pax6_cyst > cutoff_global_li_pax6)
                
                keep = keep + [cutoff_global_li_foxa2, cutoff_global_li_pax6, xx[0].size, yy[0].size]
                
                 ## cutoff of FoxA2 and Pax6 in each cyst with li thresholding
                xx = np.where(foxa2_cyst > cutoff_global_mean_foxa2) #  
                yy = np.where(pax6_cyst > cutoff_global_mean_pax6)
                
                keep = keep + [cutoff_global_mean_foxa2, cutoff_global_mean_pax6, xx[0].size, yy[0].size]
                
                
                
                df.loc[len(df)] = keep
          
        df.to_csv(os.path.join(outDir, str(fileName + "embryo_voxel_counts_global_otsu_li_mean.csv")), index=True, header=True)

250712_30xsil-041umZ_E85-embB_crop-CLE1 (193, 462, 1908) (193, 462, 1908, 4)
250712_30xsil-041umZ_E85-embB_crop-CLE2 (184, 462, 2208) (184, 462, 2208, 4)
250712_30xsil-041umZ_E85-embB_crop-somite1 (608, 462, 1366) (608, 462, 1366, 4)
250712_30xsil-041umZ_E85-embB_crop-somite2 (538, 462, 795) (538, 462, 795, 4)
250712_30xsil-041umZ_E85-embB_crop-somite3 (430, 577, 540) (430, 577, 540, 4)
250712_30xsil-041umZ_E85-embB_crop-somite4 (539, 577, 726) (539, 577, 726, 4)
250712_30xsil-041umZ_E85-embB_crop-somite5 (446, 462, 1137) (446, 462, 1137, 4)
250712_30xsil-041umZ_E85-embB_crop-somite6 (302, 462, 1398) (302, 462, 1398, 4)
250713_30xsil-041umZ_E85-embH_crop-CLE1 (207, 462, 1930) (207, 462, 1930, 4)
250713_30xsil-041umZ_E85-embH_crop-CLE2 (197, 462, 1950) (197, 462, 1950, 4)
250713_30xsil-041umZ_E85-embH_crop-somite1 (329, 462, 1875) (329, 462, 1875, 4)
250713_30xsil-041umZ_E85-embH_crop-somite2 (329, 462, 1536) (329, 462, 1536, 4)
250713_30xsil-041umZ_E85-embH_crop-somite3 (317, 462, 1368

In [ ]:
#df.to_csv(os.path.join(outDir, "embryo_voxel_counts_global_otsu_li_mean.csv"), index=True, header=True)